In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_classif
# Importing DataScience libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from scipy import stats
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler,LabelEncoder,OrdinalEncoder,power_transform
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
import warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv", encoding = "UTF-8")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv", encoding = "UTF-8")
train_c=train.copy()
test_c=test.copy()

# 1. Exploratory Data Analysis

## Objective :
- To have a better comprehension of our data
- Dévelop a first modelisation strategy

## 1.1 Basic checklist
#### First analysis :
- **variable target** : House price
- **lines / columns** : 
    * training set : 1460 / 81
    * test set : 1459 / 80



In [ ]:
print("Our traning set has the following shape : "+str(train.shape))
print("Our test set has the following shape : "+str(test.shape))

#### **Type of variables** : 

* **qualitatives** : 45 (because two variables are already int but correspond to marks on quality) 
* **quantitatives** : 36**



In [ ]:
train.dtypes.value_counts()

#### **Description of the variable**

- **PassengerId** - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.
- **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- **Destination** - The planet the passenger will be debarking to.
- **Age** - The age of the passenger.
- **VIP** - Whether the passenger has paid for special VIP service during the voyage.
- **RoomService**, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- **Name** - The first and last names of the passenger.
- **Transported** - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

## **1.2 Checking the missing values**

In [ ]:
# Checking for the Missing values
# Using isnull fuction to count the total null values in each field
total = train.isnull().sum().sort_values(ascending=False) 
# Percent of missing values is estimated by dividing total missing and the original total
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
# Concatenating the Total and Percent fields sing pandas concat fucntion
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# Displays top 20 from our max sorted list
missing_data.head(20)

**My first approach here would be :**

* Removing the different columns with more than 50% value because it won't help a lot here to predict our target variable
* We will also remove the Id column as it has nothing to do with our EDA analysis


In [ ]:
train.drop("Id",axis=1,inplace=True)
train.drop(["PoolQC","MiscFeature","Alley","Fence"],axis=1,inplace=True)

## **1.3 Checking ditribution of our target variable and independant variables**

### Target variable

In [ ]:
y = train['SalePrice']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=stats.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=stats.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=stats.lognorm)

As we can see the SalePrice does not follow normal distribution. It seems that the best fit here is the Johnson distribution and log transformation is also a very good constestant ! 

So confirmed by the kstest, our p-value is really inferior to 0.05, so we can reasonably reject the H0 hypothesis and SalePrice is not following a normal distribution.

In [ ]:
stats.kstest(np.array(train["SalePrice"]),'norm',args=(train["SalePrice"].mean(),train["SalePrice"].std()))

### Independant variable

Regarding our indepedant variable, we can check the our k-test also to verify that the other variables are not following a normal distribution.

In [ ]:
selection=train.select_dtypes(exclude=["object"]).columns

for col in selection:
    df=train[col].dropna(axis=0)
    check_var=np.array(df)
    res=stats.kstest(check_var,'norm',args=(df.mean(),df.std())).pvalue
    if res>=0.05:
        print("This column seems to be normally distirbuted : "+col+" (p_value=)"+res)
print("It's finish")    



In [ ]:
fig,ax = plt.subplots(8,5,figsize=(30,40))
i=0
#matplotlib.rcParams['figure.autolayout'] = False
for col in train.select_dtypes(exclude=['object']):
    #plt.figure() remove the line to avoid the text under the plot !
    #plt.rcParams["figure.figsize"] = (30,40)
    sns.distplot(train[col],ax=ax[i//5,i%5])
    i=i+1



As we can see here, we have also some good client for the log normal distribution. Lot Area, BsmtUnSF, TotalBsmtSF,1stFlrSF,GrLivArea. So we could apply a lognormal transformation after on those different variables.

## **1.4 The relationship between the target variable and the independant variables**

So here we will use the mutual information tool as it represents many different advantages. Indeed, Mutual information (MI) is a non-negative value that measures the mutual dependence between two random variables. The mutual information measures the amount of information we can know from one variable by observing the values of the second variable.

The mutual information is a good alternative to Pearson’s correlation coefficient, because it is able to measure any type of relationship between variables, not just linear associations. And also, it is suitable for both continuous and discrete variables, unlike Pearson’s correlation coefficient.

So first, here we will check this info between the target variables and the independant variables. So we will start our analysis with this.

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
mi_info=[]
for col in train.columns:
    t=train[[col,"SalePrice"]].dropna(axis=0)
    if col!="SalePrice":
        mi_score=make_mi_scores(t,t["SalePrice"])
        mi_info.append((col,mi_score[1]))
    else:
        pass
print("Finish")

In [ ]:
mi_scores=pd.DataFrame(mi_info,columns=["feature","score_mi"]).sort_values(by=['score_mi'],ascending=False)
plt.figure(figsize=(6, 0.25*len(mi_info)))
sns.barplot(data=mi_scores, y='feature', x='score_mi', orient='h')

As we can see, it seems that we have several variables that can be found quite logically :

GrLivArea,GareArea,LotArea, all the variables which are referring to a surfaces.

But we denote also some more interesting insights with the high contribution of the OverallQuall variable and the neighborhood also.


## Violin plot :

It could be also interesting to see in the categorical variables the distributions of SalePrices.

In [ ]:
fig,ax = plt.subplots(8,5,figsize=(30,40))
i=0
for col in train.select_dtypes('object'):
    sns.violinplot(data=train,x=train[col],y=train["SalePrice"],ax=ax[i//5,i%5])
    i=i+1

Violin graph are really interesting as it allows us to see the distribution of the variable per category but it's also interesting to the at the same time the range of our numerical variable in this category.

For example here, if we look at the ExterQual variable, we can see that houses with excellent exterior have higher prices than the others and at the same time the distribution of the sales price is more spread out for ExteriorQual==Ex.

## Box plot :

In [ ]:
fig,ax = plt.subplots(8,5,figsize=(30,40))
i=0
for col in train.select_dtypes('object'):
    sns.boxplot(data=train,x=train[col],y=train["SalePrice"],ax=ax[i//5,i%5])
    i=i+1

## **1.5 The relationship / interaction the independant variables**

In [ ]:
pd.plotting.scatter_matrix(train, figsize=(76, 76))
plt.show()

In [ ]:
plt.figure(figsize=(40,40))

corr = train.corr(method = 'spearman')

sns.heatmap(corr, annot = True)

plt.show()

So as we can see here, there are plenty of interesting relationship here ! And compared to the pearson correlation, spearman is not only working on linear relationship.

Indeed the fundamental difference between the two correlation coefficients is that the Pearson coefficient works with a linear relationship between the two variables whereas the Spearman Coefficient works with monotonic relationships as well.

So regarding the interaction, we have for example some logic ones as for example interactions between GarageYblt and Yearbuilt (as it seems logic thath Garage was built at the same time as the garage), we also have the GarageCars and the GarageArea.

## Categorical features :

It could be also interesting to use ANOVA test to check the influence of categorical variable on SalePrice. Indeed, the lower is our p-value, the higher the impact of the categorical value is. Indeed in this case, it means, that the mean between the different categories is not the same.

In [ ]:
figure,ax=plt.subplots(1,1,figsize=(50,50))
disp=[]
for c in train.select_dtypes('object'):
        samples = []
        df=train[[c,"SalePrice"]].dropna(axis=0)
        for cls in df[c].unique():
            s = df[df[c] == cls]['SalePrice'].values
            samples.append(s)
        pval = stats.f_oneway(*samples)[1]
        disp.append((c,np.log(1./pval)))

disparity=pd.DataFrame(disp,columns=["Feature","Disparity"]).sort_values(by=["Disparity"],ascending=False)
sns.barplot(data=disparity, x='Feature', y='Disparity')
ax.set_xticklabels(disparity['Feature'].values, rotation=45, ha='right',fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel('Feature', fontsize=35)
plt.ylabel('Disparity', fontsize=35)
plt.show()

**I hope you enjoyed reading this notebook ! Do not hesitate to upvote it ! :)**